# 1. Scraping
mengambil atribut sebagai berikut:
1. Judul
2. Pertanyaan
3. Penanya
4. Jawaban
5. Referensi
6. Ilustrasi

## Untuk satu post

In [1]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
    
import requests

     |████████████████████████████████| 235kB 3.4MB/s 
     |████████████████████████████████| 61kB 22.3MB/s 
     |████████████████████████████████| 3.1MB 23.1MB/s 
     |████████████████████████████████| 2.3MB 33.0MB/s 
     |████████████████████████████████| 61kB 15.3MB/s 
     |████████████████████████████████| 174kB 37.8MB/s 
     |████████████████████████████████| 112kB 42.3MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=87f61ec6bfd1eaa758efd53e65ac1fd5453a017c2d2729b04723dd569a78be21
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built PyDispatcher


In [0]:
from scrapy.http import TextResponse
from w3lib.html import remove_tags, remove_tags_with_content

URL = "https://what-if.xkcd.com/14"
response = requests.get(URL)
response = TextResponse(body=response.content, url=URL)

Sumber: https://stackoverflow.com/questions/45809023/attribute-error-response-object-has-no-attribute-css

In [0]:
def get_data(response):
  judul = response.css('h1::text').get()
  pertanyaan=[]
  penanya=[]
  if len(response.css('#question::text').getall()) > 1:
    pertanyaan.append(''.join(response.css('#question::text').getall()))
    penanya.append(''.join(response.css('#attribute::text').getall()))
  else:
    pertanyaan.append(response.css('#question::text').getall())
    penanya.append(response.css('#attribute::text').getall())
    
  jawaban = ''.join(response.css("#attribute~ p::text").extract()).replace("\n"," ").replace("\xa0"," ").replace("\'", "’")

  ref_body = []
  for ref in response.css('.ref'):
    ref_body.append(''.join(ref.css('.ref *::text').getall()))

  ilustrasi = []
  for ilustration in response.css('.illustration'):
    ilustrasi.append('https://what-if.xkcd.com' + ilustration.attrib['src'])
    
  return judul, pertanyaan, penanya, jawaban, ref_body, ilustrasi

Sumber: 
1. https://docs.scrapy.org/en/latest/intro/tutorial.html
2. 

In [4]:
judul, pertanyaan, penanya, jawaban, ref_body, ilustrasi = get_data(response)

data = {'Judul' : judul,
       'Pertanyaan' : pertanyaan,
       'Penanya' : penanya,
       'Jawaban' : jawaban,
       'Referensi' : ref_body,
       'URL_Ilustrasi' : ilustrasi}

data

{'Jawaban': 'Your brother might be surprised to learn that the water would actually make the Sun hotter!Water is made of hydrogen and oxygen, which is fuel for the Sun’s fusion. But more importantly, the extra mass also makes the Sun heavier. This crushes it together more tightly and makes fusion happen faster. This means it will burn more brightly and run through its fuel more quickly.As you keep adding water, the Sun will go through a lot of wacky fusion phases. (During one phase, called a , the reaction rate is proportional to the 40th power of the temperature—which is probably the largest exponent I’ve ever seen in a physics equation!)But one way or another, eventually the whole thing will collapse in on itself, blow off its outer layers, and become a black hole. This black hole will keep soaking up water, spraying off X-rays in the process, until finally the municipal water department notices what you’ve been doing and shuts off your service.What if you shined a flashlight (or a l

## Untuk semua post

In [0]:
URL = "https://what-if.xkcd.com/archive"
res = requests.get(URL)
res = TextResponse(body=res.content, url=URL)

In [6]:
idx = []
title = []
query = []
inquirer = []
answer = []
reference = []
ilustration = []

for articles in res.css('.archive-entry'):
  posturl = ("https:" + articles.css('a::attr(href)').get())
  r = requests.get(posturl)
  r = TextResponse(body=r.content, url=posturl)
  judul, pertanyaan, penanya, jawaban, ref_body, ilustrasi = get_data(r)
  
  title.append(judul)
  query.append(pertanyaan)
  inquirer.append(penanya)
  answer.append(jawaban)
  reference.append(ref_body)
  ilustration.append(ilustrasi)
  
  data = {'Judul' : title,
       'Pertanyaan' : query,
       'Penanya' : inquirer,
       'Jawaban' : answer,
       'Referensi' : reference,
       'URL_Ilustrasi' : ilustration}

data

{'Jawaban': ['Let’s set aside the question of how we got the baseball moving that fast. We’ll suppose it’s a normal pitch, except in the instant the pitcher releases the ball, it magically accelerates to 0.9c. From that point onward, everything proceeds according to normal physics.:The answer turns out to be “a lot of things”, and they all happen very quickly, and it doesn’t end well for the batter (or the pitcher). I sat down with some physics books, a Nolan Ryan action figure, and a bunch of videotapes of nuclear tests and tried to sort it all out. What follows is my best guess at a nanosecond-by-nanosecond portrait:The ball is going so fast that everything else is practically stationary. Even the molecules in the air are stationary. Air molecules vibrate back and forth at a few hundred miles per hour, but the ball is moving through them at 600  miles per hour. This means that as far as the ball is concerned, they’re just hanging there, frozen.The ideas of aerodynamics don’t apply he

# 2. Merapihkan Hasil

In [0]:
def get_post(response):
  judul = response.css('h1::text').get()
  
  pertanyaan=[]
  penanya=[]
  if len(response.css('#question::text').getall()) > 1:
    pertanyaan.append(''.join(response.css('#question::text').getall()))
    penanya.append(''.join(response.css('#attribute::text').getall()))
  else:
    pertanyaan.append(response.css('#question::text').getall())
    penanya.append(response.css('#attribute::text').getall())
    
  #pertanyaan = response.css('#question::text').get()
  #penanya = response.css('#attribute::text').get()
  
  jawaban = ''.join(response.css("#attribute~ p::text").extract()).replace("\n"," ").replace("\xa0"," ").replace("\'", "’")
  panjang = len(jawaban)

  ref_body = []
  pk_ref = []
  for ref in response.css('.ref'):
    pk_ref.append(ref.css('.refnum::text').get())
    ref_body.append(''.join(ref.css('.refbody *::text').getall()))

  ilustrasi = []
  name = []
  pk_ilustrasi = []
  for ilustration in response.css('.illustration'):
    pk_ilustrasi.append(ilustration.attrib['src'].split("/")[-1].split(".png")[0])
    ilustrasi.append('https://what-if.xkcd.com' + ilustration.attrib['src'])
    name.append(ilustration.attrib['title'])
    
  return judul, pertanyaan, penanya, jawaban, panjang, ref_body, ilustrasi, name, pk_ref, pk_ilustrasi

In [0]:
idx = []
title = []
query = []
inquirer = []
answer = []
reference = []
ilustration = []
ilustration_name = []
id_refer = []
id_ilus=[]
fk_ref =[]
fk_il = []
length = []
panjang_pertanyaan=[]

for articles in res.css('.archive-entry'):
  posturl = ("https:" + articles.css('a::attr(href)').get())
  id = posturl.split("/")[-2]
  r = requests.get(posturl)
  r = TextResponse(body=r.content, url=posturl)
  judul, pertanyaan, penanya, jawaban, panjang, referensi, ilustrasi, judul_ilustrasi, id_ref, id_ilustrasi = get_post(r)
  
  panjang_pertanyaan.append(len(pertanyaan))
  
  idx.append(id)
  title.append(judul)
  query.append(pertanyaan)
  inquirer.append(penanya)
  answer.append(jawaban)
  length.append(panjang)
  
  for i in range(len(referensi)):
    fk_ref.append(id)
    id_refer.append(id_ref[i] + id)
    reference.append(referensi[i])
  
  for j in range(len(ilustrasi)):
    fk_il.append(id)
    id_ilus.append(id_ilustrasi[j])
    ilustration.append(ilustrasi[j])
    ilustration_name.append(judul_ilustrasi[j])

In [0]:
Question = {'ID': idx,
           'Title' : title,
           'Query' : query,
           'Inquirer': inquirer,
           'Answer' : answer,
           'Length': length}

Ilustration = {'ID': id_ilus,
              'Question_ID': fk_il,
              'Title' : ilustration_name,
              'URL' : ilustration}

Reference = {'ID': id_refer,
              'Question_ID': fk_ref,
              'Text' : reference}

In [0]:
import pandas as pd

question = pd.DataFrame(Question)
ilustration = pd.DataFrame(Ilustration)
reference = pd.DataFrame(Reference)

In [35]:
question.head(20)

,ID,Title,Query,Inquirer,Answer,Length
0,1,Relativistic Baseball,[[What would happen if you tried to hit a base...,[[- Ellen McManis]],Let’s set aside the question of how we got the...,3955
1,2,SAT Guessing,[[What if everyone who took the SAT guessed on...,[[—Rob Balder]],None.The SAT is a standardized test given to h...,1996
2,3,Yoda,[[How much Force power can Yoda output?]],[[—Ryan Finnie]],I’m going to—of course—ignore the prequels.The...,3363
3,4,A Mole of Moles,[[What would happen if you were to gather a mo...,[[—Sean Rice]],"Things get a bit gruesome.First, some definiti...",6245
4,5,Robot Apocalypse,[[What if there was a robot apocalypse? How lo...,[[—Rob Lombino]],"Before I answer this question, let me give you...",6853
5,6,Glass Half Empty,"[[What if a glass of water was, all of a sudde...",[[—Vittorio Iacovella]],The pessimist is probably more right about how...,5163
6,7,Everybody Out,[[Is there enough energy to move the entire cu...,[[—Adam]],There are a bunch of science fiction movies wh...,6522
7,8,Everybody Jump,[[What would happen if everyone on earth stood...,[[—Thomas Bennett (and many others)]],This is one of the most popular questions subm...,3673
8,9,Soul Mates,[[What if everyone actually had only one soul ...,[[—Benjamin Staffin]],What a nightmare that would be.There are a lot...,4898
9,10,Cassini,[[What would the world be like if the land mas...,[[—Socke]],It would profoundly alter our biosphere in gen...,10145


In [36]:
ilustration.tail()

,ID,Question_ID,Title,URL
881,wait,157,"If you're reading this before May 1st, 2025, p...",https://what-if.xkcd.com/imgs/a/157/wait.png
882,shockwaves,157,SOURCE: Data from a NASA experiment/fun prank,https://what-if.xkcd.com/imgs/a/157/shockwaves...
883,parachute,157,We'll wait here. Next window is in the early 2...,https://what-if.xkcd.com/imgs/a/157/parachute.png
884,drift,157,"That's one large, supremely complicated and un...",https://what-if.xkcd.com/imgs/a/157/drift.png
885,hazard,157,"In retrospect, we could have just made the mag...",https://what-if.xkcd.com/imgs/a/157/hazard.png


Pada tabel ini terdapat beberapa link yang tidak sesuai, sebab src dari ilustrasi tersebut tidak sama dengan src yang lain

In [37]:
reference.head()

,ID,Question_ID,Text
0,[1]2,2,"Cooper, Mary Ann, MD. ""Disability, Not Death i..."
1,[2]2,2,http://www.nws.noaa.gov/om/hazstats/light08.pdf
2,[1]51,51,Super Mario 1-1 speed run
3,[2]51,51,Sprint ring cycle
4,[3]51,51,Glide data


#3. Pertanyaan

1. Tunjukan lima pertanyaan dengan jawaban terpanjang

In [39]:
question.sort_values(by=['Length'], ascending=False)[:5]

,ID,Title,Query,Inquirer,Answer,Length
9,10,Cassini,[[What would the world be like if the land mas...,[[—Socke]],It would profoundly alter our biosphere in gen...,10145
12,13,Laser Pointer,[[If every person on Earth aimed a laser point...,[[—Peter Lipowicz]],Not if we use regular laser pointers.The first...,9352
66,67,Expanding Earth,[[How long would it take for people to notice ...,[[Dennis O'Donnell]],"The Earth is not, currently, expanding.People ...",8831
156,157,Earth-Moon Fire Pole,[[My son (5y) asked me today: If there were a ...,"[[Ramon Schönborn, Germany]]","First, let’s get a few things out of the way:I...",8739
27,28,Steak Drop,[[From what height would you need to drop a st...,[[—Alex Lahey]],I hope you like your steaks Pittsburgh Rare. A...,8102


2. Berapa rata rata dan simpangan baku dari panjang jawaban?

In [40]:
print("Rata rata panjang jawaban adalah", question['Length'].mean())
print("Simpangan baku panjang jawaban adalah", question['Length'].std())

Rata rata panjang jawaban adalah 3881.923566878981
Simpangan baku panjang jawaban adalah 1573.2462826637795


3. Berapa nilai rata-rata dan simpangan baku dari jumlah referensi per pertanyaan?

In [41]:
groupby = reference.groupby('Question_ID')
print("Rata rata dari jumlah referensi per pertanyaan adalah ", groupby.size().mean())
print("Simpangan baku dari jumlah referensi per pertanyaan adalah ", groupby.size().std())

Rata rata dari jumlah referensi per pertanyaan adalah  6.076923076923077
Simpangan baku dari jumlah referensi per pertanyaan adalah  3.3895679699854973


4. Berapa nilai korelasi antara panjang pertanyaan dan panjang jawaban?

In [17]:
from scipy import stats

print ("Korelasi antara panjang pertanyaan dan panjang jawaban", stats.pearsonr(length, panjang_pertanyaan)[0])

Korelasi antara panjang pertanyaan dan panjang jawaban -0.2593551462559277


5. Berapa nilai korelasi antara panjang jawaban dan jumlah ilustrasi?

In [42]:
group_img = ilustration.groupby('Question_ID')
print ("Korelasi antara panjang pertanyaan dan panjang jawaban", stats.pearsonr(length, group_img.size())[0])

Korelasi antara panjang pertanyaan dan panjang jawaban -0.08421492122054688


#4. Bonus

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

X = question['Answer']
vec = CountVectorizer(stop_words='english').fit(X)
bow = vec.transform(X)
sum_words = np.array(bow.sum(axis=0).tolist()[0])
words = vec.get_feature_names()

words_freq = zip(words, sum_words)
words_freq = sorted(words_freq, key=lambda bow: -bow[1])

df1=pd.DataFrame(words_freq, columns =['Words', 'Sum of Word'])
df1.head(25)

,Words,Sum of Word
0,water,368
1,earth,351
2,like,270
3,probably,240
4,air,238
5,just,234
6,speed,207
7,000,197
8,time,193
9,surface,188
